### Swissmetro dataset
El conjunto de datos de swissmetro contiene preferencias declaradas para tres modos de transporte alternativos que incluyen automóvil, tren y un modo recientemente introducido: el swissmetro. 

Los datos pueden descargarse directamente desde Python utilizando el código presentado en la siguiente celda:

In [2]:
import pandas as pd
import numpy as np

df = pd.read_table("http://transp-or.epfl.ch/data/swissmetro.dat", sep='\t').dropna()
df

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,3,1,1,1192,4,1,7,1,0,5,...,148,13,30,93,17,30,0,156,56,2
10724,3,1,1,1192,4,1,7,1,0,5,...,148,12,30,96,16,10,0,96,70,3
10725,3,1,1,1192,4,1,7,1,0,5,...,148,16,60,93,16,20,0,96,56,3
10726,3,1,1,1192,4,1,7,1,0,5,...,178,16,30,96,17,30,0,96,91,2


Las variables incluyen aspectos comunes para los tres modos alternativos, como el propósito del viaje (`PURPOSE`), y variables con valores específico para cada uno, que capturan tiempo de viaje (`TT`), el _headway_ (`HE`) y el costo (`CO`). La columna `CHOICE` indica el modo elegido por el encuestado. Más detalles pueden encontrarse [acá](http://www.sze.hu/~prile/Doktori%20iskola%202018/adatok%20leiras.pdf).

Con el fin de filtrar datos que no provean aspectos relevantes para el problema presentado en la tarea, es conveniente filtrar aquellas filas donde la elección de modo sea desconocida (`CHOICE` igual a 0).

In [4]:
df = df[df['CHOICE'] != 0]
df

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,3,1,1,1192,4,1,7,1,0,5,...,148,13,30,93,17,30,0,156,56,2
10724,3,1,1,1192,4,1,7,1,0,5,...,148,12,30,96,16,10,0,96,70,3
10725,3,1,1,1192,4,1,7,1,0,5,...,148,16,60,93,16,20,0,96,56,3
10726,3,1,1,1192,4,1,7,1,0,5,...,178,16,30,96,17,30,0,96,91,2


Para los modelos con pesos compartidos, es recomendable utilizar el mismo modelo múltiples veces en la función `forward` de la red (3 veces en este caso, una por modo) y calcular la función de pérdida en base a eso. Desde el punto de vista de los datos, es recomendable manipular el `DataFrame` en el `DataLoader`, con el fin de descomponer cada fila en 3 vectores, uno por cada modo. También es posible transformar completamente el `DataFrame` con la función `wide_to_long` de Pandas, de modo de realizar la descomposición completa antes del procesamiento, pero es preferible utilizar el primer esquema por su simplicidad.